In [2]:
import pandas as pd
import numpy as np
import vectorbt as vbt

# 读取本地 data 文件夹的 BTC/USDT 1d 数据
df = pd.read_csv('data/spot/binance/1d/BTC_USDT.csv')
df['date'] = pd.to_datetime(df['datetime'])
df.set_index('date', inplace=True)

# 计算连续涨跌天数
close = df['close']
ret = close.pct_change()
up = ret > 0
down = ret < 0

up_streak = up.rolling(3).sum() == 3
down_streak = down.rolling(3).sum() == 3

# 计算3天涨跌幅
change_3d = close.pct_change(3).abs()

# 生成信号
entries = up_streak.shift(1).fillna(False)  # 连涨3天后第4天开多
exits = down_streak.shift(1).fillna(False)  # 连跌3天后第4天开空

# 止盈止损幅度
take_profit = change_3d.shift(1).fillna(0)
stop_loss = change_3d.shift(1).fillna(0)

# 用vectorbt回测
pf = vbt.Portfolio.from_signals(
    close,
    entries,
    exits,
    sl_stop=stop_loss,      # 止损百分比
    tp_stop=take_profit,    # 止盈百分比
    direction='both',       # 多空都做
    freq='1D'
)

print(pf.stats())
# 将回测结果的交互式图表保存为HTML文件，便于浏览器打开
fig = pf.plot()
fig.write_html("portfolio_result.html")
print("已保存为 portfolio_result.html，可用浏览器打开查看。")

Start                               2017-08-17 00:00:00
End                                 2025-09-07 00:00:00
Period                               2944 days 00:00:00
Start Value                                       100.0
End Value                                    178.222659
Total Return [%]                              78.222659
Benchmark Return [%]                        2493.588451
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                              66.138006
Max Drawdown Duration                1500 days 00:00:00
Total Trades                                        284
Total Closed Trades                                 283
Total Open Trades                                     1
Open Trade PnL                                      0.0
Win Rate [%]                                   47.70318
Best Trade [%]                                45.598031
Worst Trade [%]                              -25

In [ ]:
import ccxt
import pandas as pd
import datetime

# 步骤1：用ccxt获取Binance BTC/USDT 1d数据

exchange = ccxt.binance()
since = exchange.parse8601('2020-01-01T00:00:00Z')
bars = []
limit = 1000
symbol = 'BTC/USDT'
timeframe = '1d'
while True:
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since=since, limit=limit)
    if not ohlcv:
        break
    bars += ohlcv
    since = ohlcv[-1][0] + 24*60*60*1000  # 下一天
    if len(ohlcv) < limit:
        break

df = pd.DataFrame(bars, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
df['date'] = pd.to_datetime(df['timestamp'], unit='ms')
df.set_index('date', inplace=True)

In [4]:
import ccxt
from config import proxies

# 用你的OKX API信息替换下面内容
api_key = '4c7e2162-3f63-4293-8451-dcea44fb98cd'
api_secret = '9C2570C37CA9B66F0E0515D5894D493F'
api_passphrase = 'Buyao034971!'

okx = ccxt.okx({
    'apiKey': api_key,
    'secret': api_secret,
    'password': api_passphrase,
    'enableRateLimit': True,
    'proxies': proxies
})

# 切换到实盘（默认是实盘，若要测试网需加test参数）
# okx.set_sandbox_mode(True)  # 测试网用，实盘不要加


In [5]:
# check account balance
balance = okx.fetch_balance()

In [6]:
balance

{'info': {'code': '0',
  'data': [{'adjEq': '',
    'availEq': '',
    'borrowFroz': '',
    'details': [{'accAvgPx': '',
      'autoLendAmt': '0',
      'autoLendMtAmt': '0',
      'autoLendStatus': 'unsupported',
      'availBal': '162.85103910589436',
      'availEq': '162.85103910589436',
      'borrowFroz': '',
      'cashBal': '162.85103910589436',
      'ccy': 'USDT',
      'clSpotInUseAmt': '',
      'colBorrAutoConversion': '0',
      'colRes': '0',
      'collateralEnabled': False,
      'collateralRestrict': False,
      'crossLiab': '',
      'disEq': '0',
      'eq': '162.85103910589436',
      'eqUsd': '162.85429612667647',
      'fixedBal': '0',
      'frozenBal': '0',
      'imr': '0',
      'interest': '',
      'isoEq': '0',
      'isoLiab': '',
      'isoUpl': '0',
      'liab': '',
      'maxLoan': '',
      'maxSpotInUse': '',
      'mgnRatio': '',
      'mmr': '0',
      'notionalLever': '0',
      'openAvgPx': '',
      'ordFrozen': '0',
      'rewardBal': '0',
 

In [8]:

print(okx.markets['PEPE/USDT']['limits'])

{'leverage': {'min': 1.0, 'max': 10.0}, 'amount': {'min': 100000.0, 'max': None}, 'price': {'min': None, 'max': None}, 'cost': {'min': None, 'max': 1000000.0}}


In [9]:

# 下单示例：买入现货POLYDOGE/USDT，市价买入0.001 POLYDOGE
symbol = 'PEPE/USDT'
order = okx.create_market_buy_order(symbol, 100000.0)
print(order)

{'info': {'clOrdId': '6b9ad766b55dBCDEeea379f064c39c86', 'ordId': '2846954839696973824', 'sCode': '0', 'sMsg': 'Order placed', 'tag': '6b9ad766b55dBCDE', 'ts': '1757348265955'}, 'id': '2846954839696973824', 'clientOrderId': '6b9ad766b55dBCDEeea379f064c39c86', 'timestamp': None, 'datetime': None, 'lastTradeTimestamp': None, 'lastUpdateTimestamp': None, 'symbol': 'PEPE/USDT', 'type': 'market', 'timeInForce': None, 'postOnly': None, 'side': 'buy', 'price': None, 'stopLossPrice': None, 'takeProfitPrice': None, 'triggerPrice': None, 'average': None, 'cost': None, 'amount': None, 'filled': None, 'remaining': None, 'status': None, 'fee': None, 'trades': [], 'reduceOnly': False, 'fees': [], 'stopPrice': None}
